In [3]:
#from neuron_morphology.features.feature_extractor import MorphologyFeatures
import numpy as np
import pandas as pd
#import argschema as ags
import lims_utils
import layer_counts.layer_counts as lc
from PIL import Image
import urllib
import matplotlib.pyplot as plt
import seaborn as sns
import allensdk.core.swc as swc
import io
from multiprocessing import Pool
import glob
import os
import json
import matplotlib.path as mplPath
from morph_features.morph_features.features import natural_sort_key
#from neuron_morphology import swc


with open('/Users/alice/neuron_morphology_scripts/tolias_m1/mouse_m1_me_and_met_avg_layer_depths.json') as j:
    layer_depths = json.load(j)
    

In [16]:
#sps_info = pd.read_csv('/Users/alice/neuron_morphology_scripts/tolias_m1/v1/M1_V1_cell_info++.csv')
sps_info = pd.read_csv(r'/Users/alice/neuron_morphology_scripts/tolias_m1/m1_patchseq_meta_data.csv', sep='\t')
sps_info = sps_info[['Cell', 'RNA type']]
sps_info.rename(columns = {'Cell':'specimen_id'}, inplace = True)

In [17]:
histogram_df = pd.read_csv(r'/Users/alice/neuron_morphology_scripts/cell_types_card/histogram.csv')
histogram_df['specimen_id'] = [t[:-10] for t in histogram_df['specimen_id']]


In [18]:
merged_histogram = histogram_df.merge(sps_info,left_on='specimen_id',right_on='specimen_id')


all_cols_mouse = sorted(histogram_df.columns, key=natural_sort_key)
ax_cols = sorted([c for c in merged_histogram.columns if c.startswith('2_')],key=natural_sort_key)
basal_dend_cols = sorted([c for c in merged_histogram.columns if c.startswith('3_')],key=natural_sort_key)
apical_dend_cols = sorted([c for c in merged_histogram.columns if c.startswith('4_')],key=natural_sort_key)
columns_dict = {'apical':apical_dend_cols,'basal':basal_dend_cols,'axon':ax_cols}

In [19]:
def sroot(m):
    """ Returns root node of soma, if present"""
    if len(m._tree_list) > 0 and m._tree_list[0][0]['type'] == 1:
        return m._tree_list[0][0]
    return None

In [20]:
sps = []
la_depths = []
missing_sps = []
for sp in merged_histogram.specimen_id:
    #print(sp)
    sps.append(sp)
    #la_morph = '/Users/alice/neuron_morphology_scripts/tolias_m1/v1/v1_final_hist/{}_pruned.swc_connected_smoothed_marked_fixed.swc'.format(sp)
    la_morph = '/Users/alice/neuron_morphology_scripts/cell_types_card/la_swc/{}_fixed.swc'.format(sp)
    try:
        morpho = swc.read_swc(la_morph)
        la_d = abs(sroot(morpho)['y'])
        la_depths.append(la_d)
    except:
        print(sp, 'bad:', la_morph)
        la_depths.append(np.nan)
        missing_sps.append(sp)
    
depths_df = pd.DataFrame({'specimen_id':sps,'layer_aligned_depth':la_depths})
merged_histogram = merged_histogram.merge(depths_df,left_on='specimen_id',right_on='specimen_id')

In [22]:
color_dict = {'L5 ET_1' : '#0d5d7e', 'L5 ET_2': '#2686af', 'L5 ET_3': '#3aa1b6', 'L5 ET_4': '#0a75b1' , 'Pvalb Vipr2_2': '#f4358b' }

-6.793379999999988

In [24]:

uniq_labels = sorted(merged_histogram['specimen_id'].unique())

contrast_level = 0.45

for sp in uniq_labels:
    print(sp)
    this_sp = merged_histogram[merged_histogram['specimen_id']==sp]   
    ttype = this_sp.iloc[0]['RNA type']
    col = color_dict[ttype]
    axon_rgb = (tuple(int(col[1:][i:i+2], 16) for i in (0, 2, 4))) 
    axon_std = axon_rgb + (105,)
    axon_rgba = tuple([(x/float(255)) for x in axon_std])


    fig, axes = plt.subplots(3, 1, sharex=True,sharey=True, figsize=(6, 15))
    fig.suptitle(sp,y=0.98)

    outfile = '/Users/alice/neuron_morphology_scripts/cell_types_card/hist/{}.png'.format(sp)

    #col_ct = 0


    soma_depths = this_sp['layer_aligned_depth'].values
    soma_xs = np.zeros(len(soma_depths))

    row_ct=-1
    for label in ['apical','basal','axon']:
        label_columns = columns_dict[label]
        row_ct+=1
        ax = axes[row_ct]
        ax.set_ylim(0,1200)

        ax.invert_yaxis()
        #ax.set_title("{} (n={})".format(label,len(this_tt)))
        this_sp_compart = this_sp[label_columns]
        average_line = this_sp_compart.mean().values
        sem = this_sp_compart.sem().values

        lower_bound = average_line-sem
        upper_bound = average_line+sem
        lower_bound[lower_bound==0]=np.nan
        upper_bound[upper_bound==0]=np.nan
        average_line[average_line==0] = np.nan

        ax.fill_betweenx(np.arange(len(this_sp_compart.values.mean(axis=0))) * 5,lower_bound,
                 upper_bound,color=axon_rgba)

        ax.plot(average_line, np.arange(this_sp_compart.values.shape[1]) * 5., lw=2,c=col)
        ax.axhline(0, c="lightgrey")
        for v in layer_depths.values():
            ax.axhline(v, c="lightgrey")

        ax.scatter(soma_xs,soma_depths,facecolors='none', edgecolors='k')


    
    fig.savefig(outfile,dpi=300,bbox_inches='tight')
    plt.clf()

20171207_sample_2
20171219_sample_6
20180102_sample_1
20180118_sample_10
20180123_sample_7
20180225_sample_5
20190214_sample_1
20190619_sample_5
20190926_sample_2
20191030_sample_3


<Figure size 432x1080 with 0 Axes>

<Figure size 432x1080 with 0 Axes>

<Figure size 432x1080 with 0 Axes>

<Figure size 432x1080 with 0 Axes>

...

<Figure size 432x1080 with 0 Axes>

<Figure size 432x1080 with 0 Axes>

<Figure size 432x1080 with 0 Axes>

<Figure size 432x1080 with 0 Axes>

<Figure size 432x1080 with 0 Axes>